In [ ]:
# import the modules
import os
import GPy
import sys
import numpy as np
from matplotlib import cm
import cPickle as pickle
import scipy.stats as stats
import sklearn.metrics as metrics
import GPy.plotting.Tango as Tango
from tqdm import tqdm_notebook as tqdm
from matplotlib import pyplot as plt
from sklearn.decomposition import PCA
from GPy.plotting.matplot_dep.controllers.imshow_controller import ImshowController

In [ ]:
%matplotlib notebook

In [ ]:
PROJECT_DIR = 'HSI2020/data'
DATA_DIR = os.path.join(PROJECT_DIR, 'proc')
JOINT_ANGLES_DIR = os.path.join(DATA_DIR, 'Baxter')
WHILL_DIR = os.path.join(DATA_DIR, 'Whill')

In [ ]:
def get_data(file_index):
    JOINT_ANGLES_FILE = os.path.join(JOINT_ANGLES_DIR, 'joint_angles_%d.csv' % file_index)
    WHILL_FILE = os.path.join(WHILL_DIR, 'whill_movement_%d.csv' % file_index)

    # load the joint angles train data
    joint_angles = np.loadtxt(JOINT_ANGLES_FILE, delimiter=',', skiprows=1)

    # load the whill movement train data
    whill_movement = np.loadtxt(WHILL_FILE, delimiter=',', skiprows=1)

    # remove time stamp info from both observations
    joint_angles = joint_angles[:, 1:]
    whill_movement = whill_movement[:, 1:]
    return joint_angles, whill_movement

In [ ]:
# define the index of the files used for testing and training
train_files = [3]
test_files = [1, 2]

In [ ]:
joint_angles = []
whill_movement = []
for idx in train_files:
    FILE_INDEX = idx
    ja, wm = get_data(FILE_INDEX)
    joint_angles.append(ja)
    whill_movement.append(wm)

joint_angles = np.concatenate(joint_angles, axis=0)
whill_movement = np.concatenate(whill_movement, axis=0)

In [ ]:
joint_angles.shape

In [ ]:
whill_movement.shape

In [ ]:
plt.figure()
for idx in range(len(train_files)):
    i = 200 * idx
    j = 200 * (idx + 1)
    plt.plot(whill_movement[i:j,0])

In [ ]:
joint_angles.shape

In [ ]:
whill_movement.shape

In [ ]:
# add small bias
bias = 1e-4 * np.random.normal(size=whill_movement.shape)
whill_movement += bias
whill_movement.shape

In [ ]:
rep_cols = 5
whill_movement = np.tile(whill_movement, reps=(1, rep_cols))
whill_movement.shape

In [ ]:
def plotScales(scales1, scales2, options, yThresh=0.05):
    fig = plt.figure()    
    ax = fig.add_subplot(111)
    
    x = np.arange(1,scales1.shape[0] + 1)
    c1 = Tango.colorsHex['mediumBlue']
    c2 = Tango.colorsHex['darkGreen']
    h1 = ax.bar(x, height=scales1, width=0.8, align='center', color=c1, linewidth=1.3)
    h2 = ax.bar(x, height=scales2, width=0.5, align='center', color=c2, linewidth=0.7)
    ax.plot([0.4, scales1.shape[0] + 0.6], [yThresh, yThresh], '--', linewidth=3, color=Tango.colorsHex['mediumRed'])
    
    # setting the bar plot parameters
    ax.set_xlim(.4, scales1.shape[0]+.6)
    ax.tick_params(axis='both')
    ax.set_xticks(xrange(1,scales1.shape[0]+1))
    ax.set_title(options['title'])
    ax.set_ylabel(options['ylabel'])
    ax.set_xlabel('Latent Dimensions')
    ax.legend([h1,h2],options['labels'], loc='upper right')

In [ ]:
def plotLatent(inX,
               title,
               model=None,
               which_indices=[0,1], 
               plot_inducing=False, 
               plot_variance=False, 
               max_points=[800, 300],
               test_also=False):
    s = 100
    marker = 'o'    
    resolution = 50
    
    fig = plt.figure()
    ax = fig.add_subplot(111)
    Tango.reset()

    input1, input2 = which_indices

    if inX[0].shape[0] > max_points[0]:
        print("Warning".format(inX[0].shape))
        subsample = np.random.choice(inX[0].shape[0], size=max_points[0], replace=False)
        inX[0] = inX[0][subsample]

    if test_also:
        if inX[1].shape[0] > max_points[1]:
            print("Warning".format(inX[1].shape))
            subsample = np.random.choice(inX[1].shape[0], size=max_points[1], replace=False)
            inX[1] = inX[1][subsample]
        
    xmin, ymin = inX[0][:, [input1, input2]].min(0)
    xmax, ymax = inX[0][:, [input1, input2]].max(0)
    x_r, y_r = xmax-xmin, ymax-ymin
    xmin -= .1*x_r
    xmax += .1*x_r
    ymin -= .1*y_r
    ymax += .1*y_r
    print xmin, xmax, ymin, ymax
    
    if plot_variance:
        def plotFunction(x):
            Xtest_full = np.zeros((x.shape[0], qDim))
            Xtest_full[:, [input1, input2]] = x
            _, var = model.predict(np.atleast_2d(Xtest_full))
            var = var[:, :1]
            return -np.log(var)
        qDim = model.X.mean.shape[1]
        x, y = np.mgrid[xmin:xmax:1j*resolution, ymin:ymax:1j*resolution]
        gridData = np.hstack((x.flatten()[:, None], y.flatten()[:, None]))
        gridVariance = (plotFunction(gridData)).reshape((resolution, resolution))
        varianceHandle = plt.imshow(gridVariance.T, interpolation='bilinear', origin='lower', cmap=cm.gray, extent=(xmin, xmax, ymin, ymax))
        
    trainH = ax.scatter(inX[0][:, input1], inX[0][:, input2], marker=marker, s=s, c=Tango.colorsHex['mediumBlue'], linewidth=.2, alpha=1.)
    
    if test_also:
        testH = ax.scatter(inX[1][:, input1], inX[1][:, input2], marker=marker, s=s, c=Tango.colorsHex['mediumRed'], linewidth=.2, alpha=0.9)
    
    ax.grid(b=False) 
    ax.set_aspect('auto')
    ax.tick_params(axis='both')

    if test_also:
        ax.legend([trainH,testH],['Train','Test'],loc='upper right')
    else:
        ax.legend([trainH],['Train'], loc='upper right')
        
    ax.set_xlabel('Latent Dimension %i' % (input1 + 1))
    ax.set_ylabel('Latent Dimension %i' % (input2 + 1))
    
    if plot_inducing:
        Z = model.Z
        ax.scatter(Z[:, input1], Z[:, input2], c='w', s=25, marker="^", linewidth=.3, alpha=.6)

    ax.set_xlim((xmin, xmax))
    ax.set_ylim((ymin, ymax))

In [ ]:
# set the overall dimensions for MRD
latent_dim = 8

# out of 8 dimensions, 6 dimensions are assigned to baxter's joint angle
joint_angle_dim = 6

num_inducing = 100

dim_values = [range(joint_angle_dim), range(joint_angle_dim, latent_dim)]

train_list = [joint_angles, whill_movement] # list of input and corresponding label

In [ ]:
# optimization variables
SNR0 = 100
SNR1 = 50
train_iters = 200
cons_mod0_max_iters = 200
cons_mod1_max_iters = 200
fix_var_max_iters = 6

In [ ]:
num_samples = train_list[0].shape[0]

scales = []
input_X = np.zeros((num_samples, latent_dim))
dim_distribution = [joint_angle_dim, latent_dim - joint_angle_dim]

for dim, values, Y in zip(dim_distribution, dim_values, train_list):
    X, var = GPy.util.initialization.initialize_latent('PCA', dim, Y)
    scales.extend(var)
    input_X[:, values] = X

scales = np.asarray(scales)

mrd_kernels = [GPy.kern.RBF(latent_dim, variance=1, lengthscale=1.0/scales, ARD=True) for _ in train_list]

mrd_model = GPy.models.MRD(train_list,
                           input_dim=latent_dim,
                           num_inducing=num_inducing,
                           kernel=mrd_kernels,
                           X=input_X,
                           name='mrd')

In [ ]:
# Phase 1: Optimizaition by fixing variance parameters
var0 = mrd_model.Y0.Y.var()
var1 = mrd_model.Y1.Y.var()

mrd_model.Y0.rbf.variance.fix(var0)
mrd_model.Y1.rbf.variance.fix(var1)

mrd_model.Y0.Gaussian_noise.variance.fix(var0 / SNR0)
mrd_model.Y1.Gaussian_noise.variance.fix(var1 / SNR1)

mrd_model.optimize(messages=True, max_iters=fix_var_max_iters)

print 'Phase 1 Optimizaition Done.'

In [ ]:
# Phase 2.1: Optimize each model individually
# constrain space 0
mrd_model.Y1.constrain_fixed()
mrd_model.optimize(messages=True, max_iters=cons_mod0_max_iters)

print 'Phase 2.1 Optimizaition Done.'

In [ ]:
# Phase 2.2: Optimize each model individually
# constrain space 1
mrd_model.Y0.constrain_fixed()
mrd_model.Y1.unconstrain_fixed()
mrd_model.Y1.rbf.variance.fix(var1)
mrd_model.Y1.Gaussian_noise.variance.fix(var1 / SNR1)
mrd_model.optimize(messages=True, max_iters=cons_mod1_max_iters)
    
print 'Phase 2.2 Optimizaition Done.'

In [ ]:
# Phase 3: Optimize the model without any constraints
# training without constraints
mrd_model.Y0.unconstrain_fixed()
mrd_model.Y1.unconstrain_fixed()
mrd_model.optimize(messages=True, max_iters=train_iters)
    
print 'Phase 3 Optimizaition Done.'

In [ ]:
scales1 = mrd_model.Y0.kern.input_sensitivity(summarize=False)
scales2 = mrd_model.Y1.kern.input_sensitivity(summarize=False)

scales1 /= scales1.max()
scales2 /= scales2.max()

options = {'title':'ARD Weights','ylabel':'ARD Weight','labels':['JointAngle', 'WhillMovement']}
plotScales(scales1, scales2, options)

plt.savefig('ARD_Weights_Baxter_to_Whill.pdf', format='pdf')

In [ ]:
pickle.dump(mrd_model, open('mrd_model.pkl','wb'))

In [ ]:
# function to compute reconstruction error
def reconstructionErrorMRD(model, valData, testData, mKey, kKey, optimizeFlag=False):    
    nSamplesVal = valData[mKey].shape[0]
    nSamplesTest = testData[mKey].shape[0]

    dims = val_data[mKey].shape[1]  
    nDimIn = valData[kKey].shape[1]
    nDimOut = dims
    
    qDim = model.X.mean.shape[1]
    
    # computing reconstruction error for test1, test2 with variances
    predictVal = np.zeros((nSamplesVal,nDimOut))
    predictTest = np.zeros((nSamplesTest,nDimOut))

    latentVal = np.zeros((nSamplesVal,qDim))
    latentTest = np.zeros((nSamplesTest,qDim))

    for n in tqdm(range(nSamplesVal)):
        yIn = valData[kKey][n,:]
        yTrueOut = valData[mKey][n]
    
        [xPredict, infX] = model.Y0.infer_newX(yIn[None,:], optimize=False)
        yOut = model.predict(xPredict.mean, Yindex=1)    
        #sys.stdout.write('.')
        
        predictVal[n,:] = yOut[0]
        latentVal[n,:] = xPredict.mean
        
    #sys.stdout.write('\n\n')
        
    for n in tqdm(range(nSamplesTest)):
        yIn = testData[kKey][n,:]
        yTrueOut = testData[mKey][n]
    
        [xPredict, infX] = model.Y0.infer_newX(yIn[None,:], optimize=optimizeFlag)
        yOut = model.predict(xPredict.mean, Yindex=1)    
        #sys.stdout.write('.')
        
        predictTest[n,:] = yOut[0]
        latentTest[n,:] = xPredict.mean
        
    #sys.stdout.write('\n\n')
    results = {}
    valResults = {}
    testResults = {}
    
    valResults['pred'] = predictVal
    testResults['pred'] = predictTest
    
    valResults['latent'] = latentVal
    testResults['latent'] = latentTest
    
    valErrors = np.sqrt(metrics.mean_squared_error(valData[mKey],predictVal,multioutput='raw_values'))
    testErrors = np.sqrt(metrics.mean_squared_error(testData[mKey],predictTest,multioutput='raw_values'))

    valNormErrors = np.divide(np.sqrt(metrics.mean_squared_error(valData[mKey],predictVal,multioutput='raw_values')), 
                              valData[mKey].max(axis=0) - valData[mKey].min(axis=0))
    testNormErrors = np.divide(np.sqrt(metrics.mean_squared_error(testData[mKey],predictTest,multioutput='raw_values')), 
                               testData[mKey].max(axis=0) - testData[mKey].min(axis=0))

    valCorr = np.zeros((1,nDimOut))
    testCorr = np.zeros((1,nDimOut))
    for d in range(dims):
        valCorr[0,d],_ = stats.pearsonr(valData[mKey][1],predictVal[d])
        testCorr[0,d],_ = stats.pearsonr(testData[mKey][1],predictTest[d])

    valResults['rmse'] = valErrors
    testResults['rmse'] = testErrors
    
    valResults['nrmse'] = valNormErrors
    testResults['nrmse'] = testNormErrors
    
    valResults['corr'] = valCorr
    testResults['corr'] = testCorr
        
    results['train'] = valResults
    results['test'] = testResults
    return results

In [ ]:
mrd_model = pickle.load(open('mrd_model.pkl','rb'))
train_X = mrd_model.X.mean
mrd_X = [train_X, train_X]

plotLatent(mrd_X, 
           'Latent Space', 
           model=mrd_model, 
           which_indices=[0, 1], 
           plot_variance=True, 
           max_points=[300, 300], 
           test_also=False, 
           plot_inducing=False)